### **В этом блокноте мы конвертируем записи из списков блокадников (простой текст) в табличку, извлекая из них информацию регулярками и костылями.**

In [1]:
import os
import re
from bs4 import BeautifulSoup as beat_soup
import pandas as pd

In [2]:
PATH_TO_HTMLS = "./blocade_base/" # путь к папке с хтмлками
DEATH_DATE = r"смерти: [1,2]?[0-9]? ?\w{3,10} 19[0-9]{2}" # смертельная регулярка

Достаём пути ко всем хтмлкам.

In [3]:
htmls = []

for _, dirs, files in os.walk(PATH_TO_HTMLS):
    for file in files:
        htmls.append(PATH_TO_HTMLS + file)  

Готовим функции для извлечения текста из хтмл и извлечения информации из собственно текста.

In [4]:
def get_text(html_path: str) -> str:
    """ достаем текстовую статью с хтмл-странички """
    with open(html_path, "r") as f:    
        contents = f.read() 
        soup = beat_soup(contents, 'lxml')
        
    # странички сделаны странно и теги у них не всегда одинаковы - пробуем оба варианта
    if soup.find("div", class_ = "col-sm-12"):
        return soup.find("div", class_ = "col-sm-12").text.strip().replace("\n", "")
    return soup.find("div", class_ = "col-sm-9").text.strip().replace("\n", "")

In [5]:
def get_person_info(text: str) -> dict:
    """ создаем словарь с информацией о человеке """
    person = text 
    info = {"fio": "no_info", "birth_date": "no_info",
            "address": "no_info", "death_date": "no_info", "burial_place": "no_info",
            "source": "no_info"}
    
    fio = person.split(",")[0] # чаще всего отделено запятой, но иногда нет... :
    if "Место" in fio:
        fio = fio.split("Место")[0].strip()
    if "Дата" in fio:
        fio = fio.split("Дата")[0].strip() 
    
    try:
        birth_date = re.findall(r"[0-3]?[0-9]?.?[0-1]?[0-9]?.?1[8-9][0-9]{2}", 
                                person.split("г.р.")[0].split("род")[1])[0].strip("., ")  
        # (да. выше было что-то уродливое. но оно... типа... работает)
    except:
        birth_date = "no_info" # иногда даты рождения не бывает
        
    info["fio"], info["birth_date"] = fio, birth_date
    
    # дергаем место проживания, если есть
    if "Место проживания" in person:  
        addr = person.split("Место проживания: ")[1] 
        if "Дата смерти" in person:
            address = addr.split("Дата")[0].strip(". ")
        else:
            address = addr.split("(")[0].strip(". ")
            info["address"] = address
     
    # дергаем всякое про смерть, если есть (чаще нет)
    if "Дата смерти" in person: 
        if re.findall(DEATH_DATE, person):
            death_date = re.findall(DEATH_DATE, person)[0].replace("смерти: ", "")
            info["death_date"] = death_date 
            
    if "Место захоронения" in person:
        burial_place = person.split("Место захоронения: ")[-1].split("(")[0]
        info["burial_place"] = burial_place
    
    # источник достается не всегда -- иногда на страничке он просто в другом блоке
    # это надо дописать
    if "(" in person:
        source = person.split("(")[-1].strip(") ")
        info["source"] = source
    
    return info

Обходим все хтмлки и собираем информацию для каждой из них + добавляем айдишники.

In [6]:
dicts = []

for x in htmls:    
    try: # была какая-то странная ошибка с одним из файлов, закостылила
        x_info = get_person_info(get_text(x))
        x_info["id"] = x.split("/")[-1].strip(".html")
    except:
        print (x) 
    dicts.append(x_info)    

./blocade_base/._906849.html


Собираем табличку.

In [7]:
blockade_table = pd.DataFrame(columns = dicts[0].keys())

In [8]:
for n in blockade_table: 
    blockade_table[n] = [x[n] for x in dicts]

In [9]:
blockade_table

,fio,birth_date,address,death_date,burial_place,source,id
0,Аксакова-Акинчис Наталья Александровна,03.05.1882,"г. Ленинград, Таврическая ул., д. 9, кв. 22",no_info,no_info,"Они пережили Блокаду, т. 1",1587194
1,Аак Алида Ивановна,05.04.1922,Московский р-н,no_info,no_info,"Они пережили Блокаду, т. 1",1584263
2,Абакшина Лидия Павловна,no_info,no_info,февраль 1942,Серафимовское кладб.,no_info,906974
3,Алатырев Анатолий Михайлович,28.11.1938,Невский р-н,no_info,no_info,"Они пережили Блокаду, т. 1",1587667
4,Абезгауз Нинель Всеволодовна,no_info,no_info,апрель 1942,неизвестно.,no_info,907143
5,Авдеева Маргарита Анатольевна,16.03.1938,Красногвардейский р-н,no_info,no_info,"Они пережили Блокаду, т. 1",1585314
6,Агеева Галина Александровна,25.11.1938,г. Петродворец,no_info,no_info,"Они пережили Блокаду, т. 1",1586162
7,Агафонова Людмила Дмитриевна,19.08.1938,Приморский р-н,no_info,no_info,"Они пережили Блокаду, т. 1",1586063
8,Авербах Раиса Моисеевна,07.03.1915,Красногвардейский р-н,no_info,no_info,"Они пережили Блокаду, т. 1",1585413
9,Александрова Валентина Павловна,10.12.1936,Невский р-н,no_info,no_info,"Они пережили Блокаду, т. 1",1588218


In [10]:
# выгружаем в файлик
blockade_table.to_csv("blockade_example.csv", sep = "\t")